<a href="https://colab.research.google.com/github/EdwSanA/DPro_Tareas/blob/main/Red_neuronal_profunda_ajustado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# ===============================
# Scratch Deep Neural Network - Full Implementation
# Problemas 1 al 9
# ===============================

import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# ==================================================
# [Problema 1] Capa totalmente conectada (FC)
# ==================================================
class FC:
    def __init__(self, n_nodes1, n_nodes2, initializer, optimizer):
        self.W = initializer.W(n_nodes1, n_nodes2)
        self.B = initializer.B(n_nodes2)
        self.optimizer = optimizer

    def forward(self, X):
        self.X = X
        A = np.dot(X, self.W) + self.B
        return A

    def backward(self, dA):
        self.dW = np.dot(self.X.T, dA) / self.X.shape[0]
        self.dB = np.mean(dA, axis=0)
        dZ = np.dot(dA, self.W.T)
        self = self.optimizer.update(self)
        return dZ

# ==================================================
# [Problema 2] Inicialización Simple
# ==================================================
class SimpleInitializer:
    def __init__(self, sigma=0.01):
        self.sigma = sigma

    def W(self, n_nodes1, n_nodes2):
        return self.sigma * np.random.randn(n_nodes1, n_nodes2)

    def B(self, n_nodes2):
        return np.zeros(n_nodes2)

# ==================================================
# [Problema 3] Optimización con SGD
# ==================================================
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr

    def update(self, layer):
        layer.W -= self.lr * layer.dW
        layer.B -= self.lr * layer.dB
        return layer

# ==================================================
# [Problema 4] Funciones de Activación
# ==================================================
class Tanh:
    def forward(self, X):
        self.Z = np.tanh(X)
        return self.Z

    def backward(self, dA):
        return dA * (1 - self.Z**2)

class Softmax:
    def forward(self, X):
        X = X - np.max(X, axis=1, keepdims=True)
        exp_X = np.exp(X)
        self.Z = exp_X / np.sum(exp_X, axis=1, keepdims=True)
        return self.Z

    def backward(self, Z, Y):
        batch_size = Y.shape[0]
        return (Z - Y) / batch_size

# ==================================================
# [Problema 5] ReLU
# ==================================================
class ReLU:
    def forward(self, X):
        self.X = X
        return np.maximum(0, X)

    def backward(self, dA):
        dZ = dA * (self.X > 0)
        return dZ

# ==================================================
# [Problema 6] Xavier y He Initializers
# ==================================================
class XavierInitializer:
    def W(self, n_nodes1, n_nodes2):
        sigma = 1 / np.sqrt(n_nodes1)
        return sigma * np.random.randn(n_nodes1, n_nodes2)

    def B(self, n_nodes2):
        return np.zeros(n_nodes2)

class HeInitializer:
    def W(self, n_nodes1, n_nodes2):
        sigma = np.sqrt(2 / n_nodes1)
        return sigma * np.random.randn(n_nodes1, n_nodes2)

    def B(self, n_nodes2):
        return np.zeros(n_nodes2)

# ==================================================
# [Problema 7] AdaGrad
# ==================================================
class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h_W, self.h_B = None, None

    def update(self, layer):
        if self.h_W is None:
            self.h_W = np.zeros_like(layer.W)
            self.h_B = np.zeros_like(layer.B)

        self.h_W += layer.dW * layer.dW
        self.h_B += layer.dB * layer.dB

        layer.W -= self.lr * layer.dW / (np.sqrt(self.h_W) + 1e-7)
        layer.B -= self.lr * layer.dB / (np.sqrt(self.h_B) + 1e-7)
        return layer

# ==================================================
# [Problema 8] Clase ScratchDeepNeuralNetrowkClassifier
# ==================================================
class ScratchDeepNeuralNetrowkClassifier:
    def __init__(self, n_epochs=10, batch_size=32, verbose=True):
        self.n_epochs = n_epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.layers = []

    def add_fc(self, input_dim, output_dim, activation="relu", initializer="simple", optimizer="sgd", lr=0.01):
        # Inicializador
        if initializer == "simple":
            init = SimpleInitializer(0.01)
        elif initializer == "xavier":
            init = XavierInitializer()
        elif initializer == "he":
            init = HeInitializer()
        else:
            init = SimpleInitializer(0.01)

        # Optimizador
        if optimizer == "sgd":
            opt = SGD(lr)
        elif optimizer == "adagrad":
            opt = AdaGrad(lr)
        else:
            opt = SGD(lr)

        fc = FC(input_dim, output_dim, init, opt)
        self.layers.append(fc)

        # Activación
        if activation == "relu":
            self.layers.append(ReLU())
        elif activation == "tanh":
            self.layers.append(Tanh())
        elif activation == "softmax":
            self.layers.append(Softmax())

    def fit(self, X, y, X_val=None, y_val=None):
        for epoch in range(self.n_epochs):
            idx = np.random.permutation(len(X))
            X_shuffled, y_shuffled = X[idx], y[idx]

            for i in range(0, len(X), self.batch_size):
                X_batch = X_shuffled[i:i+self.batch_size]
                y_batch = y_shuffled[i:i+self.batch_size]

                # Forward
                A = X_batch
                for layer in self.layers:
                    if isinstance(layer, Softmax):
                        A = layer.forward(A)
                    else:
                        A = layer.forward(A)

                # Backward
                dA = self.layers[-1].backward(A, y_batch)
                for layer in reversed(self.layers[:-1]):
                    dA = layer.backward(dA)

            if self.verbose:
                y_pred = self.predict(X_val)
                acc = np.mean(y_pred == np.argmax(y_val, axis=1))
                print(f"Epoch {epoch+1}/{self.n_epochs} - Precisión: {acc:.4f}")

    def predict(self, X):
        A = X
        for layer in self.layers:
            if isinstance(layer, Softmax):
                A = layer.forward(A)
            else:
                A = layer.forward(A)
        return np.argmax(A, axis=1)

# ==================================================
# [Problema 9] Entrenamiento y Validación con MNIST
# ==================================================
print("Descargando MNIST...")
mnist = fetch_openml('mnist_784', version=1, as_frame=False)
X = mnist.data.astype(np.float32) / 255.0
y = mnist.target.astype(int)

X_train, X_valid, y_train, y_valid = train_test_split(
    X[:10000], y[:10000], test_size=0.2, random_state=42
)

enc = OneHotEncoder(sparse_output=False)
Y_train_oh = enc.fit_transform(y_train.reshape(-1,1))
Y_valid_oh = enc.transform(y_valid.reshape(-1,1))

net = ScratchDeepNeuralNetrowkClassifier(n_epochs=5, batch_size=64, verbose=True)
net.add_fc(input_dim=784, output_dim=128, activation="relu", initializer="he", optimizer="adagrad", lr=0.01)
net.add_fc(input_dim=128, output_dim=64, activation="tanh", initializer="xavier", optimizer="adagrad", lr=0.01)
#net.ad

Descargando MNIST...


In [3]:
# Definición de la red
net = ScratchDeepNeuralNetrowkClassifier(
    n_epochs=5, batch_size=64, verbose=True
)

net.add_fc(input_dim=784, output_dim=128, activation="relu", initializer="he", optimizer="adagrad", lr=0.01)
net.add_fc(input_dim=128, output_dim=64, activation="tanh", initializer="xavier", optimizer="adagrad", lr=0.01)
net.add_fc(input_dim=64, output_dim=10, activation="softmax", initializer="xavier", optimizer="adagrad", lr=0.01)

# Entrenamiento
print("Entrenando red neuronal...")
net.fit(X_train, Y_train_oh, X_valid, Y_valid_oh)

# Predicciones
y_pred = net.predict(X_valid)

# Precisión
acc = np.mean(y_pred == y_valid)
print(f"Precisión final en validación: {acc:.4f}")


Entrenando red neuronal...
Epoch 1/5 - Precisión: 0.9255
Epoch 2/5 - Precisión: 0.9410
Epoch 3/5 - Precisión: 0.9470
Epoch 4/5 - Precisión: 0.9500
Epoch 5/5 - Precisión: 0.9470
Precisión final en validación: 0.9470
